# tshark

In [1]:
!tshark --autostop packets:20

Running as user "root" and group "root". This could be dangerous.
Capturing on 'wlp2s0'
    1 0.000000000 AVMAudio_3b:b4:a6 → Broadcast    HomePlug AV 60 Qualcomm Atheros, GET_SW.REQ (Get Device/SW Version Request)
    2 0.000256009 AVMAudio_3b:b4:a6 → Broadcast    0x8912 60 Ethernet II
    3 1.536230877 wlan0.fritz.box → 255.255.255.255 UDP 214 54311 → 6667 Len=172
    4 1.854462226 xps.fritz.box → wpad.fritz.box DNS 92 Standard query 0x61e0 A mobile.events.data.microsoft.com
    5 1.854986634 xps.fritz.box → wpad.fritz.box DNS 92 Standard query 0x10c7 AAAA mobile.events.data.microsoft.com
    6 1.861692264 wpad.fritz.box → xps.fritz.box DNS 213 Standard query response 0x61e0 A mobile.events.data.microsoft.com CNAME mobile.events.data.trafficmanager.net CNAME onedscolprduks03.uksouth.cloudapp.azure.com A 51.105.71.137
    7 1.862291357 wpad.fritz.box → xps.fritz.box DNS 268 Standard query response 0x10c7 AAAA mobile.events.data.microsoft.com CNAME mobile.events.data.trafficmanager.net

#### remote unixoid machines
To capture traffic you may want to do so on a remote intermittent machine (router, switch, etc). If this is not dedicated monitoring hardware, like a managed switch or similar, you can `pcap` with tshark (fixed or continuous)
```bash
# manual / autocapture
tshark -w /tmp/ten-seconds.pcap --autostop duration:10
# write continuously, 
tshark -w /tmp/one-day.pcap --ring-buffer duration:86400
ls -lah | grep one-day.pcap
```

In [4]:
!tshark -r run8.pcap -T json > run8.json

Running as user "root" and group "root". This could be dangerous.


In [30]:
import json
global capture
with open('run8.json') as cap:
    capture = json.load(cap)

modbus_packets = [
    {'ip.src': packet['_source']['layers']['ip']['ip.src'],
     'ip.dst': packet['_source']['layers']['ip']['ip.dst'],
     'modbus':packet['_source']['layers']['modbus']}
    for packet in capture
    if 'modbus' in packet['_source']['layers']]

modbus_packets

[{'ip.src': '192.168.1.100',
  'ip.dst': '192.168.1.102',
  'modbus': {'modbus.func_code': '3',
   'modbus.reference_num': '8',
   'modbus.word_cnt': '4'}},
 {'ip.src': '192.168.1.102',
  'ip.dst': '192.168.1.100',
  'modbus': {'modbus.func_code': '3',
   'modbus.request_frame': '4',
   'modbus.response_time': '0.000838000',
   'modbus.byte_cnt': '8',
   'Register 8 (UINT16): 12000': {'modbus.regnum16': '8',
    'modbus.regval_uint16': '12000'},
   'Register 9 (UINT16): 0': {'modbus.regnum16': '9',
    'modbus.regval_uint16': '0'},
   'Register 10 (UINT16): 0': {'modbus.regnum16': '10',
    'modbus.regval_uint16': '0'},
   'Register 11 (UINT16): 0': {'modbus.regnum16': '11',
    'modbus.regval_uint16': '0'}}},
 {'ip.src': '192.168.1.100',
  'ip.dst': '192.168.1.101',
  'modbus': {'modbus.func_code': '3',
   'modbus.reference_num': '8',
   'modbus.word_cnt': '4'}},
 {'ip.src': '192.168.1.101',
  'ip.dst': '192.168.1.100',
  'modbus': {'modbus.func_code': '3',
   'modbus.request_frame': 

In [39]:
frequencies = {}
keystr = lambda k, p: ":".join([k, p[k]])
increment = lambda m, k: m[k] + 1 if k in m else 1
for packet in modbus_packets:
    frequencies[keystr('ip.src', packet)] = increment(frequencies, keystr('ip.src', packet))
    frequencies[keystr('ip.dst', packet)] = increment(frequencies, keystr('ip.dst', packet))
    frequencies[keystr('modbus.func_code', packet['modbus'])] = increment(frequencies, keystr('modbus.func_code', packet['modbus']))

dict(sorted(frequencies.items()))

{'ip.dst:192.168.1.100': 3258,
 'ip.dst:192.168.1.101': 2171,
 'ip.dst:192.168.1.102': 2173,
 'ip.dst:192.168.1.103': 2167,
 'ip.dst:192.168.1.99': 3253,
 'ip.src:192.168.1.100': 3258,
 'ip.src:192.168.1.101': 2171,
 'ip.src:192.168.1.102': 2173,
 'ip.src:192.168.1.103': 2167,
 'ip.src:192.168.1.99': 3253,
 'modbus.func_code:1': 4272,
 'modbus.func_code:2': 4274,
 'modbus.func_code:3': 4278,
 'modbus.func_code:5': 198}

# sharkd

In [3]:
!echo '{"req": "info"}' | sharkd -

Running as user "root" and group "root". This could be dangerous.
Hello in child.
{"columns":[{"name":"802.1Q VLAN id","format":"%q"},{"name":"Absolute date, as YYYY-MM-DD, and time","format":"%Yt"},{"name":"Absolute date, as YYYY/DOY, and time","format":"%YDOYt"},{"name":"Absolute time","format":"%At"},{"name":"Cisco VSAN","format":"%V"},{"name":"Cumulative Bytes","format":"%B"},{"name":"Custom","format":"%Cus"},{"name":"DCE/RPC call (cn_call_id / dg_seqnum)","format":"%y"},{"name":"Delta time","format":"%Tt"},{"name":"Delta time displayed","format":"%Gt"},{"name":"Dest addr (resolved)","format":"%rd"},{"name":"Dest addr (unresolved)","format":"%ud"},{"name":"Dest port (resolved)","format":"%rD"},{"name":"Dest port (unresolved)","format":"%uD"},{"name":"Destination address","format":"%d"},{"name":"Destination port","format":"%D"},{"name":"Expert Info Severity","format":"%a"},{"name":"FW-1 monitor if/direction","format":"%I"},{"name":"Frequency/Channel","format":"%F"},{"name":"Hardware

# Scapy

### Install

In [ ]:
!pip3 install scapy

## Basics

In [4]:
from scapy.all import IP

# create a basic IP-Packet
IP().show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = hopopt
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



### Monitoring
#### a simple network scanner
To monitor which devices are on your network, you may want to actively search for active IPs

In [12]:
!ip address

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: wlp2s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether c4:8e:8f:f3:9b:a7 brd ff:ff:ff:ff:ff:ff
    inet 192.168.178.22/24 brd 192.168.178.255 scope global dynamic noprefixroute wlp2s0
       valid_lft 860978sec preferred_lft 860978sec
    inet6 2001:4dd3:8f2a:0:8c55:12e8:cf1a:ffbb/64 scope global temporary dynamic 
       valid_lft 6937sec preferred_lft 3337sec
    inet6 2001:4dd3:8f2a:0:2a1d:3830:67ba:c18e/64 scope global dynamic mngtmpaddr noprefixroute 
       valid_lft 6937sec preferred_lft 3337sec
    inet6 fe80::470:6b4c:4354:bc76/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [5]:
# Are you root?
# Make sure you have elevated privileges for the next step
!whoami

root


In [6]:
# src: https://www.thepythoncode.com/article/building-networks-scanner-using-scapy
from scapy.all import ARP, Ether, srp
# set target IP range
target = "192.168.0.1/24" # your local subnet
# create ARP (Address Resolution protocol) packet
arp = ARP(pdst=target)
ether = Ether(dst="ff:ff:ff:ff:ff:ff")
packet = ether/arp

# raw socket operations require elevated privileges
result = srp(packet, timeout=10) # => Tuple (answered, unanswered)

print("===== Query Answer =====")
print(result[0][0])

print("===== Devices =====")
{received.psrc: received.hwsrc for sent, received in result[0]}

Begin emission:
Finished sending 256 packets.
*******..............
Received 21 packets, got 7 answers, remaining 249 packets
===== Query Answer =====
QueryAnswer(query=<Ether  dst=ff:ff:ff:ff:ff:ff type=ARP |<ARP  pdst=192.168.178.1 |>>, answer=<Ether  dst=c4:8e:8f:f3:9b:a7 src=3c:37:12:3b:b4:a6 type=ARP |<ARP  hwtype=0x1 ptype=IPv4 hwlen=6 plen=4 op=is-at hwsrc=3c:37:12:3b:b4:a6 psrc=192.168.178.1 hwdst=c4:8e:8f:f3:9b:a7 pdst=192.168.178.22 |>>)
===== Devices =====


{'192.168.178.1': '3c:37:12:3b:b4:a6',
 '192.168.178.20': '60:45:cb:9a:3a:bd',
 '192.168.178.29': '00:f6:20:80:9c:75',
 '192.168.178.38': '34:ab:95:1c:00:d3',
 '192.168.178.39': 'c4:5b:be:e4:47:eb',
 '192.168.178.40': '18:69:d8:c9:1d:0e',
 '192.168.178.36': 'c8:c9:a3:cb:34:54'}

#### a simple network sniffer

In [ ]:
# adapted from src: https://www.thepythoncode.com/article/dhcp-listener-using-scapy-in-python
from scapy.all import sniff, Ether, DHCP

global print_packet, target_mac, requested_ip, hostname, vendor_id

def listen():
    sniff(prn=print_packet, iface='wlp2s0')

def print_packet(packet):
    print(packet.summary())
    #print(packet.show())

listen()

#### network sniffer "pcap style"

In [ ]:
from scapy.all import *

pcap = sniff(offline='', )